In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
固定讀取指定 Result.txt，計算相鄰兩筆位移量 (6 位小數)，並輸出 CSV。
執行：
    python compute_displacement_fixed.py
"""

import re
import math
import csv
from pathlib import Path

# ===== 1. 指定檔案路徑 ==========================================
TXT_FILE = Path("/media/dc0206/Crucial X6/GMM20/20240329_DATA/NTHU_5x/red06/Result.txt")
# ==============================================================

# 欲採用的欄位
X_COL = "AeroCmdX"
Y_COL = "AeroCmdY"

PAIR_RE = re.compile(r"([\w\d]+):\s*([-+]?[0-9]*\.?[0-9]+)")

def parse_line(line: str):
    return {k: float(v) for k, v in PAIR_RE.findall(line)}

def main():
    if not TXT_FILE.exists():
        raise FileNotFoundError(f"找不到檔案：{TXT_FILE}")

    rows = []
    with TXT_FILE.open(encoding="utf-8", errors="ignore") as f:
        for line in f:
            if line.strip():
                rows.append(parse_line(line))
    if not rows:
        raise ValueError("檔案內容無可解析資料")

    # --- 計算 ΔX, ΔY, ΔDist ----------------------------------
    prev_x = prev_y = None
    for r in rows:
        cur_x, cur_y = r.get(X_COL), r.get(Y_COL)
        if prev_x is None:
            dx = dy = dist = 0.0
        else:
            dx, dy = cur_x - prev_x, cur_y - prev_y
            dist = math.hypot(dx, dy)
        # 🔸 四捨五入至 6 位
        r.update(
            DeltaX=round(dx, 6),
            DeltaY=round(dy, 6),
            DeltaDist=round(dist, 6)
        )
        prev_x, prev_y = cur_x, cur_y

    # --- 找出位移為 0 的紀錄 ---------------------------------
    zero_moves = []
    for idx, r in enumerate(rows, start=1):
        if math.isclose(r["DeltaDist"], 0.0, abs_tol=1e-10):   # 🔸
            zero_moves.append((idx, r))

    if zero_moves:
        print("\n=== 位移 = 0 (六位精度) ===")
        for idx, r in zero_moves:
            if "ImgName" in r:
                print(f"- 行 {idx}: {r['ImgName']}")
            else:
                print(f"- 行 {idx}: DeltaDist = 0")
        print("================================\n")
    else:
        print("\n⚠️  沒有 DeltaDist = 0 的紀錄\n")

    # --- 輸出 CSV (固定 6 位小數) -----------------------------
    out_path = TXT_FILE.with_name(TXT_FILE.stem + "_with_disp.csv")
    fieldnames = rows[0].keys()
    with out_path.open("w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            # 🔸 轉成格式化字串，確保 6 位
            r_fmt = {k: (f"{v:.6f}" if isinstance(v, float) else v) for k, v in r.items()}
            writer.writerow(r_fmt)

    print(f"✅ 完成！輸出：{out_path}")

if __name__ == "__main__":
    main()



=== 位移 = 0 (六位精度) ===
- 行 1: DeltaDist = 0
- 行 11: DeltaDist = 0
- 行 14: DeltaDist = 0
- 行 15: DeltaDist = 0
- 行 21: DeltaDist = 0
- 行 24: DeltaDist = 0
- 行 30: DeltaDist = 0
- 行 31: DeltaDist = 0
- 行 32: DeltaDist = 0
- 行 33: DeltaDist = 0
- 行 41: DeltaDist = 0
- 行 44: DeltaDist = 0
- 行 47: DeltaDist = 0
- 行 50: DeltaDist = 0
- 行 51: DeltaDist = 0
- 行 57: DeltaDist = 0
- 行 58: DeltaDist = 0
- 行 64: DeltaDist = 0
- 行 69: DeltaDist = 0
- 行 70: DeltaDist = 0
- 行 71: DeltaDist = 0
- 行 72: DeltaDist = 0
- 行 73: DeltaDist = 0
- 行 74: DeltaDist = 0
- 行 77: DeltaDist = 0
- 行 96: DeltaDist = 0
- 行 99: DeltaDist = 0
- 行 102: DeltaDist = 0
- 行 105: DeltaDist = 0
- 行 109: DeltaDist = 0
- 行 110: DeltaDist = 0
- 行 113: DeltaDist = 0
- 行 114: DeltaDist = 0
- 行 118: DeltaDist = 0
- 行 119: DeltaDist = 0

✅ 完成！輸出：/media/dc0206/Crucial X6/GMM20/20240329_DATA/NTHU_5x/red06/Result_with_disp.csv


In [1]:
"""多資料集點雲‑影像對齊批次處理程式
------------------------------------------------
使用方式：
1. 先在 DATASETS 清單中填入每個資料集的路徑與檔名設定。
2. 執行 python multi_dataset_registration.py
   -> 程式會依序處理所有資料夾，結果各自輸出為 Excel。
   -> 若某資料集失敗，錯誤訊息會列出，但不影響其他批次。

程式大綱：
    ├─ 演算法函式（補點、DBSCAN 過濾、USAC、ICP/CPD...）
    ├─ process_dataset(cfg): 處理單一資料集
    └─ main(): 迴圈呼叫 process_dataset()
"""

# === 0. 套件 ===
import os, glob, time
import numpy as np, pandas as pd, cv2
import matplotlib.pyplot as plt
from typing import Dict, List
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from pycpd import RigidRegistration  # pip install pycpd

# ---------------------------
# 1. 補點函式
# ---------------------------

def ensure_two_points_in_circle_all(filtered_points, radius: float = 10.0):
    """確保每個圓形區域內至少兩點，如不足則補點"""
    if len(filtered_points) < 1:
        return filtered_points, []

    new_points = []
    for i, center in enumerate(filtered_points):
        dists = np.linalg.norm(filtered_points - center, axis=1)
        in_circle_indices = np.where(dists <= radius)[0]
        if len(in_circle_indices) < 2:
            if len(filtered_points) == 1:
                rand_dir = np.random.randn(2)
                rand_dir /= (np.linalg.norm(rand_dir) + 1e-9)
                new_pt = center + rand_dir * (radius * 0.5)
                new_points.append(new_pt)
            else:
                sorted_idx = np.argsort(dists)
                nearest_idx = sorted_idx[1] if sorted_idx[0] == i else sorted_idx[0]
                nearest_point = filtered_points[nearest_idx]
                direction_vec = center - nearest_point
                dist_np = np.linalg.norm(direction_vec)
                if dist_np < 1e-9:
                    direction_vec = np.random.randn(2)
                    dist_np = np.linalg.norm(direction_vec)
                dir_unit = direction_vec / dist_np
                new_pt = center + dir_unit * (radius * 0.8)
                new_points.append(new_pt)

    if len(new_points) > 0:
        new_points = np.array(new_points)
        updated_points = np.vstack([filtered_points, new_points])
    else:
        updated_points = filtered_points
    return updated_points, new_points

def ensure_two_points_in_circle_iterative(points, radius: float = 10.0, max_iter: int = 100):
    updated_points = points.copy()
    for _ in range(max_iter):
        updated_points, new_points = ensure_two_points_in_circle_all(updated_points, radius=radius)
        if len(new_points) == 0:
            break
    return updated_points

# ---------------------------
# 2. 雜訊過濾與輔助函式
# ---------------------------

def filter_noise_points_weighted(points, eps: float = 3, min_samples: int = 2,
                                 min_cluster_size: int = 55, return_mask: bool = False):
    if len(points) == 0:
        return np.array([]), np.array([]) if not return_mask else (np.array([]), np.array([]), None)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(points)
    labels = db.labels_
    unique_labels, counts = np.unique(labels[labels != -1], return_counts=True)
    large_clusters = set(unique_labels[counts >= min_cluster_size])
    mask = np.isin(labels, list(large_clusters))
    filtered_points = points[mask]
    return (filtered_points, mask) if not return_mask else (filtered_points, mask, mask)

def compute_local_density(points, k: int = 18):
    if len(points) == 0:
        return np.array([])
    actual_k = min(k + 1, len(points))
    nbrs = NearestNeighbors(n_neighbors=actual_k).fit(points)
    distances, _ = nbrs.kneighbors(points)
    avg_distance = np.mean(distances[:, 1:], axis=1)
    density = 1.0 / (avg_distance + 1e-6)
    return density

def estimate_rigid_transform(src, dst):
    centroid_src = np.mean(src, axis=0)
    centroid_dst = np.mean(dst, axis=0)
    src_centered = src - centroid_src
    dst_centered = dst - centroid_dst
    H = src_centered.T @ dst_centered
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[1, :] *= -1
        R = Vt.T @ U.T
    t = centroid_dst - R @ centroid_src
    return R, t

def usac_rigid_transform(src_points, dst_points, *, num_iterations: int = 100,
                         inlier_threshold: float = 50.0, min_inliers: int = 60):
    best_inlier_count = 0
    best_R = best_t = best_inlier_mask = None
    N = src_points.shape[0]
    if N < 2:
        raise ValueError("來源點數不足")

    for _ in range(num_iterations):
        if N < 8:
            break
        indices = np.random.choice(N, 8, replace=False)
        src_sample = src_points[indices]
        dst_sample = dst_points[indices]
        if np.linalg.norm(src_sample[1] - src_sample[0]) < 1e-10:
            continue
        R_cand, t_cand = estimate_rigid_transform(src_sample, dst_sample)
        src_transformed = (R_cand @ src_points.T).T + t_cand
        distances = np.linalg.norm(dst_points - src_transformed, axis=1)
        inlier_mask = distances < inlier_threshold
        inlier_count = np.sum(inlier_mask)
        if inlier_count > best_inlier_count and inlier_count >= min_inliers:
            best_inlier_count = inlier_count
            best_R, best_t, best_inlier_mask = R_cand, t_cand, inlier_mask
            break
    if best_inlier_mask is None or np.sum(best_inlier_mask) < min_inliers:
        raise ValueError("USAC 未找到足夠內點")
    src_inliers = src_points[best_inlier_mask]
    dst_inliers = dst_points[best_inlier_mask]
    best_R, best_t = estimate_rigid_transform(src_inliers, dst_inliers)
    return best_R, best_t, best_inlier_mask

# ---------------------------
# 3. ICP / CPD / Normal‑ICP
# ---------------------------

def icp_with_weights(source_points, target_points, weights, *, max_iterations: int = 500, tolerance: float = 1e-5):#-20
    R_total = np.eye(2)
    t_total = np.zeros(2)
    prev_error = float("inf")
    for iteration in range(max_iterations):
        transformed_source = (R_total @ source_points.T).T + t_total
        nbrs = NearestNeighbors(n_neighbors=1).fit(target_points)
        distances, indices = nbrs.kneighbors(transformed_source)
        closest_points = target_points[indices.flatten()]
        error = np.sum(weights * (distances.flatten() ** 2)) / (np.sum(weights) + 1e-9)
        if abs(prev_error - error) < tolerance and error < 1:
            break
        prev_error = error
        src_centroid = np.average(transformed_source, axis=0, weights=weights)
        tgt_centroid = np.average(closest_points, axis=0, weights=weights)
        src_cent = transformed_source - src_centroid
        tgt_cent = closest_points - tgt_centroid
        H = (weights[:, None] * src_cent).T @ tgt_cent
        U, _, Vt = np.linalg.svd(H)
        R_delta = Vt.T @ U.T
        if np.linalg.det(R_delta) < 0:
            Vt[1, :] *= -1
            R_delta = Vt.T @ U.T
        t_delta = tgt_centroid - R_delta @ src_centroid
        R_total = R_delta @ R_total
        t_total = R_delta @ t_total + t_delta
    return R_total, t_total, prev_error

def cpd_rigid_registration(source_points, target_points, *, max_iterations: int = 50, tolerance: float = 1e-3):
    reg = RigidRegistration(X=target_points, Y=source_points,
                            max_iterations=max_iterations, tolerance=tolerance)
    Y_reg, (s, R, t) = reg.register()
    return s, R, t, Y_reg

def compute_normals(points, k: int = 40):
    if len(points) < 3:
        return np.zeros_like(points)
    actual_k = min(k, len(points) - 1)
    nbrs = NearestNeighbors(n_neighbors=actual_k).fit(points)
    _, indices = nbrs.kneighbors(points)
    normals = np.zeros_like(points)
    for i, neighbors in enumerate(indices):
        cov_matrix = np.cov(points[neighbors].T)
        eigvals, eigvecs = np.linalg.eigh(cov_matrix)
        normals[i] = eigvecs[:, 0]
    return normals / (np.linalg.norm(normals, axis=1, keepdims=True) + 1e-9)

def normal_icp(source_points, target_points, weights, *, alpha: float = 0.1, gamma: float = 0.8,
               max_iterations: int = 500, tolerance: float = 1e-5):
    source_normals = compute_normals(source_points)
    target_normals = compute_normals(target_points)
    R_total = np.eye(2)
    t_total = np.zeros(2)
    prev_error = float("inf")
    for iteration in range(max_iterations):
        transformed_source = (R_total @ source_points.T).T + t_total
        nbrs = NearestNeighbors(n_neighbors=1).fit(target_points)
        distances, indices = nbrs.kneighbors(transformed_source)
        indices = np.clip(indices.flatten(), 0, len(target_normals) - 1)
        closest_points = target_points[indices]
        closest_normals = target_normals[indices]
        normal_dot = np.einsum("ij,ij->i", source_normals, closest_normals)
        normal_errors = np.arccos(np.clip(normal_dot, -1, 1))
        weighted_normals = np.exp(-normal_errors / (2 * alpha)) ** gamma
        combined_weights = weights * distances.flatten() * weighted_normals
        error = np.sum(combined_weights * (distances.flatten() ** 2)) / (np.sum(combined_weights) + 1e-9)
        if abs(prev_error - error) < tolerance and error < 1:
            break
        prev_error = error
        src_centroid = np.average(transformed_source, axis=0, weights=combined_weights)
        tgt_centroid = np.average(closest_points, axis=0, weights=combined_weights)
        src_cent = transformed_source - src_centroid
        tgt_cent = closest_points - tgt_centroid
        H = (combined_weights[:, None] * src_cent).T @ tgt_cent
        U, _, Vt = np.linalg.svd(H)
        R_delta = Vt.T @ U.T
        if np.linalg.det(R_delta) < 0:
            Vt[1, :] *= -1
            R_delta = Vt.T @ U.T
        t_delta = tgt_centroid - R_delta @ src_centroid
        R_total = R_delta @ R_total
        t_total = R_delta @ t_total + t_delta
    return R_total, t_total, prev_error

def fill_line_gaps(points, *, max_distance: float = 3.0, max_iter: int = 5):
    pts = points.copy()
    for _ in range(max_iter):
        nbrs = NearestNeighbors(n_neighbors=2).fit(pts)
        distances, indices = nbrs.kneighbors(pts)
        new_pts = []
        for i in range(len(pts)):
            d = distances[i, 1]
            if d > max_distance:
                p1, p2 = pts[i], pts[indices[i, 1]]
                num_insert = int(d // max_distance)
                for n in range(1, num_insert + 1):
                    new_pts.append(p1 + (p2 - p1) * n / (num_insert + 1))
        if not new_pts:
            break
        pts = np.vstack([pts, np.array(new_pts)])
    return pts

"""{
        "name": "red01",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red01",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red01",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red01/Image_20240321100710525.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321100710525.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red01_combined.xlsx",
    },
    {
        "name": "red02",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red02",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red02",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red02/Image_20240321104026349.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321104026349.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red02_combined.xlsx",
    },
    {
        "name": "red03",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red03",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red03",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red03/Image_20240321105545542.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321105545542.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red03_combined.xlsx",
    },
    {
        "name": "red04",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red04",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red04",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red04/Image_20240321111055812.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321111055812.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red04_combined.xlsx",
    },
    {
        "name": "red05",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red05",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red05",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red05/Image_20240321112738655.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321112738655.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red05_combined.xlsx",
    },
    {
        "name": "red06",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red06",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red06",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red06/Image_20240321114408009.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321114408009.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red06_combined.xlsx",
    },
"""
# ---------------------------
# 4. DATASETS 參數清單
# ---------------------------
DATASETS: List[Dict] = [
{
        "name": "red01",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red01",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red01",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red01/Image_20240321100710525.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321100710525.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red01_combined.xlsx",
    },
    {
        "name": "red02",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red02",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red02",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red02/Image_20240321104026349.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321104026349.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red02_combined.xlsx",
    },
    {
        "name": "red03",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red03",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red03",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red03/Image_20240321105545542.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321105545542.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red03_combined.xlsx",
    },
    {
        "name": "red04",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red04",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red04",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red04/Image_20240321111055812.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321111055812.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red04_combined.xlsx",
    },
    {
        "name": "red05",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red05",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red05",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red05/Image_20240321112738655.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321112738655.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red05_combined.xlsx",
    },
    {
        "name": "red06",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red06",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/red06",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/red06/Image_20240321114408009.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321114408009.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/red06_combined.xlsx",
    },
    {
        "name": "white01",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white01",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white01",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white01/Image_20240321143412883.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321143412883.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white01_combined.xlsx",
    },
    {
        "name": "white02",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white02",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white02",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white02/Image_20240321145312162.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321145312162.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white02_combined.xlsx",
    },
    {
        "name": "white03",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white03",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white03",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white03/Image_20240321151040617.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321151040617.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white03_combined.xlsx",
    },
    {
        "name": "white04",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white04",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white04",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white04/Image_20240321152853920.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321152853920.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white04_combined.xlsx",
    },
    {
        "name": "white05",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white05",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white05",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white05/Image_20240321154611642.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321154611642.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white05_combined.xlsx",
    },
    {
        "name": "white06",
        "csv_dir":  r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white06",
        "img_dir":  r"/media/dc0206/Crucial X6/GMM20/20240329_DATA2/20240329_DATA/NTHU_5x/white06",
        "template_csv": r"/media/dc0206/Crucial X6/GMM20/EdgeFinderResult/white06/Image_20240321160945507.csv",
        "template_img": r"/media/dc0206/Crucial X6/GMM20/crop_w5.bmp",
        "first_img": "Image_20240321160945507.bmp",
        "final_xlsx": r"/media/dc0206/Crucial X6/GMM20/white06_combined.xlsx",
    },

    # --- 新資料集請直接在此處繼續加入 ------------------------------------
]



In [6]:
# 5. 單資料集完整流程

def process_dataset(cfg: Dict):
    start = time.time()
    # === 讀 cfg ===
    csv_directory     = cfg["csv_dir"]
    image_directory   = cfg["img_dir"]
    template_csv_path = cfg["template_csv"]
    template_path     = cfg["template_img"]
    first_img         = cfg["first_img"]
    final_output      = cfg["final_xlsx"]

    # === 步驟耗時統計 ===
    step_times = {
        "template": [],
        "csv_read": [],
        "denoise": [],
        "usac": [],
        "icp": [],
    }

# === 1. 第一張影像建立參考 ===
# === 0. 讀取模板 ===
    t0 = time.time()
    template_data = pd.read_csv(template_csv_path)
    template_points = template_data[['PosX','PosY']].values
    first_image = cv2.imread(os.path.join(image_directory, first_img), cv2.IMREAD_GRAYSCALE)
    template_img = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    res_match = cv2.matchTemplate(first_image, template_img, cv2.TM_CCOEFF_NORMED)
    _, _, _, first_tl = cv2.minMaxLoc(res_match)
    first_br = ( first_tl[0] + template_img.shape[1],
                first_tl[1] + template_img.shape[0] )
    step_times["template"].append(time.time() - t0)

    # 篩選模板區域
    template_pts_filtered = template_points[
        (template_points[:,0] >= first_tl[0]) & (template_points[:,0] <= first_br[0]) &
        (template_points[:,1] >= first_tl[1]) & (template_points[:,1] <= first_br[1])
    ]

    h,w = template_img.shape[:2]
    # 用第一次的 first_tl 算正中間
    region_center = np.array([ first_tl[0] + w/2.0,
                            first_tl[1] + h/2.0 ])
    if len(template_pts_filtered) < 3:
        raise ValueError("模板區域點不足")
    
    # === 1. 第一張影像建立參考 ===
    t0 = time.time()
    df_first = pd.read_csv(os.path.join(csv_directory, first_img.replace('.bmp','.csv')))
    pts1 = df_first[['PosX','PosY']].values
    cond1 = (
        (pts1[:,0] >= first_tl[0]) & (pts1[:,0] <= first_br[0]) &
        (pts1[:,1] >= first_tl[1]) & (pts1[:,1] <= first_br[1])
    )
    first_matched = pts1[cond1]
    # 只取過濾後的點
    ref = filter_noise_points_weighted(first_matched)[0]
    ref = ensure_two_points_in_circle_iterative(ref, radius=0.5, max_iter=300)
    ref = fill_line_gaps(ref, max_distance=2.0, max_iter=500)
    step_times["csv_read"].append(time.time() - t0)

    # 初始化 results 與 prev_inc
    results = []
    prev_inc = np.zeros(2)

    # 第一張直接存
    results.append({
        "image_file": first_img,
        "rotation_matrix": np.eye(2).tolist(),
        "translation_vector": [0,0],
        "incremental_displacement": [0,0],
        "incremental_disp_norm": 0.0,
        "displacement": [0,0],
        "displacement_norm": 0.0,
        "alignment_error": 0.0,
    })

    # --- 迴圈處理其餘影像 ---
    imgs = sorted(glob.glob(os.path.join(image_directory, '*.bmp')))
    csvs = sorted(glob.glob(os.path.join(csv_directory, '*.csv')))
    for idx in range(1, len(imgs)):
        # 2-1 模板匹配找 tl, center
        t0 = time.time()
        im = cv2.imread(imgs[idx], cv2.IMREAD_GRAYSCALE)
        res = cv2.matchTemplate(im, template_img, cv2.TM_CCOEFF_NORMED)
        _, _, _, tl = cv2.minMaxLoc(res)
        step_times["template"].append(time.time() - t0)
        # 區域中心
        region_center = np.array([ tl[0] + w/2.0,
                                   tl[1] + h/2.0 ])

        # 2-2 讀 CSV + 篩點
        t0 = time.time()
        dfc = pd.read_csv(csvs[idx])
        pts = dfc[['PosX','PosY']].values
        cond = ((pts[:,0]>=tl[0])&(pts[:,0]<=tl[0]+w)&
                (pts[:,1]>=tl[1])&(pts[:,1]<=tl[1]+h))
        cur = pts[cond]
        step_times["csv_read"].append(time.time() - t0)
        if len(cur)<3 or len(ref)<3:
            continue

        # 2-3 雜訊過濾 + 補點
        t0 = time.time()
        # 只取過濾後的點
        cur = filter_noise_points_weighted(cur)[0]
        cur = ensure_two_points_in_circle_iterative(cur, radius=0.5, max_iter=300)
        cur = fill_line_gaps(cur, max_distance=2.0, max_iter=500)
        step_times["denoise"].append(time.time() - t0)

        # 2-4 權重
        dens = compute_local_density(cur, k=25)
        dens = (dens - dens.min())/(dens.max()-dens.min()+1e-9)

        # 2-5 USAC
        t0 = time.time()
        nbrs = NearestNeighbors(n_neighbors=1).fit(ref)
        _,idx2 = nbrs.kneighbors(cur)
        tgt = ref[idx2.flatten()]
        try:
            R_u, t_u, _ = usac_rigid_transform(cur, tgt, inlier_threshold=40, min_inliers=20)
        except:
            R_u, t_u = np.eye(2), np.zeros(2)
        step_times["usac"].append(time.time() - t0)

        # 2-6 ICP + CPD + Normal-ICP
        t0 = time.time()
        R_p, t_p, e_p = icp_with_weights(cur, ref, dens)
        try:
            _, R_c, t_c, _ = cpd_rigid_registration(cur, ref, max_iterations=100, tolerance=1e-3)
        except:
            R_c, t_c = np.eye(2), np.zeros(2)
        R_n, t_n, e_n = normal_icp((R_c@cur.T).T + t_c, ref, dens)
        if e_p < e_n:
            R_f, t_f, err_f = R_p, t_p, e_p
        else:
            R_f = R_n @ R_c
            t_f = R_n @ t_c + t_n
            err_f = e_n
        step_times["icp"].append(time.time() - t0)
        #print(region_center)
        # 2-7 計算增量位移：考慮旋轉+平移的 region_center
        transformed_center = R_f @ region_center + t_f
        #transformed_center = region_center + t_f
        inc = transformed_center - region_center
        disp = inc - prev_inc
        prev_inc = inc.copy()

        # 2-8 收集結果
        results.append({
            "image_file": os.path.basename(imgs[idx]),
            "rotation_matrix":          R_f.tolist(),
            "translation_vector":       t_f.tolist(),
            "incremental_displacement": inc.tolist(),
            "incremental_disp_norm":    float(np.linalg.norm(inc)),
            "displacement":             disp.tolist(),
            "displacement_norm":        float(np.linalg.norm(disp)),
            "alignment_error":          float(err_f),
        })


    # === 3. 輸出 Excel ===
    os.makedirs(os.path.dirname(final_output), exist_ok=True)
    pd.DataFrame(results).to_excel(final_output, index=False)
    print(f"✅ [{cfg['name']}] 完成，檔案：{final_output}，總耗時 {(time.time()-start):.1f}s")
    # === 4. 畫步驟耗時圓餅圖 ===
    labels = list(step_times.keys())
    totals = [sum(step_times[k]) for k in labels]
    if sum(totals)>0:
        plt.figure(figsize=(6,6))
        plt.pie(totals, labels=labels, autopct="%1.1f%%", startangle=45)
        plt.axis("equal")
        chart_dir = os.path.join(os.path.dirname(final_output),"time_charts")
        os.makedirs(chart_dir, exist_ok=True)
        pie_path = os.path.join(chart_dir, f"{cfg['name']}_time_pie.png")
        plt.savefig(pie_path, dpi=300, bbox_inches="tight")
        plt.close()

    print(f"✅ [{cfg['name']}] 完成，檔案：{final_output}，總耗時 {(time.time()-start):.1f}s")

def main():
    for cfg in DATASETS:
        try:
            process_dataset(cfg)
        except Exception as e:
            print(f"[{cfg['name']}] 失敗：{e}")

if __name__ == "__main__":
    main()



✅ [red01] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red01_combined.xlsx，總耗時 42.5s
✅ [red01] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red01_combined.xlsx，總耗時 42.6s
✅ [red02] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red02_combined.xlsx，總耗時 46.3s
✅ [red02] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red02_combined.xlsx，總耗時 46.4s
✅ [red03] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red03_combined.xlsx，總耗時 27.4s
✅ [red03] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red03_combined.xlsx，總耗時 27.5s
✅ [red04] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red04_combined.xlsx，總耗時 29.4s
✅ [red04] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red04_combined.xlsx，總耗時 29.4s
✅ [red05] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red05_combined.xlsx，總耗時 28.1s
✅ [red05] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red05_combined.xlsx，總耗時 28.2s
✅ [red06] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red06_combined.xlsx，總耗時 28.7s
✅ [red06] 完成，檔案：/media/dc0206/Crucial X6/GMM20/red06_combined.xlsx，總耗時 28.7s
✅ [white01] 完成，檔案：/media/dc0206/Crucial X6/GMM20/white01_combined.xlsx，總耗時 3